In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [22]:
from pathlib import Path

print(Path.cwd())  # /home/skovorodkin/stack

/workspaces/src/trader_buddy/jupyter-notebooks


In [3]:
import yfinance as yf

In [4]:
sp500 = yf.Ticker("IVV")

In [6]:
sp500.info["shortName"]

'iShares Core S&P 500 ETF'

In [10]:
sp500.info["uuid"]

'17732e4a-0401-33ba-977d-a9848ab99a1e'

In [11]:
sp500.info["exchange"]

'PCX'

In [12]:
sp500.info["open"]

489.94

In [13]:
sp500.info["bid"]

489.62

In [16]:
sp500.info["previousClose"]

487.61

In [17]:
sp500.info["volume"]

6635596

In [9]:
list(sp500.info)

['phone',
 'longBusinessSummary',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'trailingPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'yield',
 'totalAssets',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'trailingAnnualDividendRate',
 'trailingAnnualDividendYield',
 'navPrice',
 'currency',
 'category',
 'ytdReturn',
 'beta3Year',
 'fundFamily',
 'fundInceptionDate',
 'legalType',
 'threeYearAverageReturn',
 'fiveYearAverageReturn',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'trailingPegRatio']

In [18]:
# get historical market data
# hist = sp500.history(period="1mo")
hist = sp500.history(period="1day")

In [19]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2024-01-25 00:00:00-05:00,489.940002,490.619995,487.839996,490.329987,6615725,0.0,0.0,0.0
